# 任务 5：使用批量转换从大型数据集获取推理

## 任务 5.1：环境设置

安装程序包和依赖项。

In [ ]:
#install-dependencies
import boto3
import sagemaker
import sagemaker_datawrangler
import time
from sagemaker.session import Session
from sagemaker.transformer import Transformer

role = sagemaker.get_execution_role()
region = boto3.Session().region_name
sess = boto3.Session()
sm = sess.client('sagemaker')
prefix = 'sagemaker/mlasms'
sagemaker_session = sagemaker.Session()
bucket = sagemaker.Session().default_bucket()
s3_client = boto3.client("s3")

将来自训练和优化实验的模型保存在默认 Amazon Simple Storage Service (Amazon S3) 存储桶中。使用 **create_model** 设置模型，并配置 **ModelDataUrl** 以引用经过训练的模型。

In [ ]:
#set-up-model
# Upload the model and dataset to your Amazon S3 bucket
s3_client.upload_file(Filename="model.tar.gz", Bucket=bucket, Key=f"{prefix}/models/model.tar.gz")

# Set a date to use in the model name
create_date = time.strftime("%Y-%m-%d-%H-%M-%S")
model_name = 'income-model-{}'.format(create_date)

# Retrieve the container image
container = sagemaker.image_uris.retrieve(
    region=boto3.Session().region_name, 
    framework='xgboost', 
    version='1.5-1'
)

# Set up the model
income_model = sm.create_model(
    ModelName = model_name,
    ExecutionRoleArn = role,
    PrimaryContainer = {
        'Image': container,
        'ModelDataUrl': f's3://{bucket}/{prefix}/models/model.tar.gz',
    }
)

将批量记录上传到默认 Amazon S3 存储桶。

In [ ]:
#upload-dataset
s3_client.upload_file(Filename="batch_data.csv", Bucket=bucket, Key=f"{prefix}/batch_data.csv", ExtraArgs={"ContentType": "text/csv;charset=utf-8"})
batch_path = f"s3://{bucket}/{prefix}/batch_data.csv"

## 任务 5.2：创建批量转换任务

批量转换会在指定参数的限制内自动管理大型数据集的处理。批量转换任务启动时，SageMaker 会初始化计算实例，并在这些实例之间分配推理或预处理工作负载。批量转换按键对输入中的 Amazon S3 对象进行分区，并将 Amazon S3 对象映射到实例。

如果您需要从大型数据集获取推理或不需要持久性终端节点，请使用批量转换。

要创建批量转换任务，您需要设置以下选项：
- **model_name**：模型的名称。
- **instance_type**：要使用的 Amazon Elastic Compute Cloud (Amazon EC2) 实例的类型；例如“ml.c4.xlarge”。
- **instance_count**：要使用的 EC2 实例数。
- **assemble_with**：输出的组合方式。有效值为“Line”或“None”。
- **strategy**：用于确定如何在单个请求中批处理记录的策略。有效值为“MultiRecord”和“SingleRecord”。
- **accept**：接受的文件类型。
- **output_path**：用于保存转换结果的 Amazon S3 位置。如果未指定，结果将存储到默认存储桶。

In [ ]:
#create-batch-transformer
transformer = Transformer(
    model_name=model_name,
    instance_type="ml.m4.xlarge",
    instance_count=1,
    assemble_with="Line",
    strategy="MultiRecord",
    accept="text/csv",
    output_path="s3://{}/{}/batch-transform/test".format(bucket, prefix)
)

使用测试数据集作为客户记录并运行批量转换任务。使用这组客户记录运行该任务大约需要长达 10 分钟的时间。

有关批量转换任务的更多信息，请参阅 [Use Batch Transform](https://docs.aws.amazon.com/sagemaker/latest/dg/batch-transform.html)。

In [ ]:
#run-batch-transform-job
transformer.transform(batch_path, content_type="text/csv", split_type="Line", join_source="Input")
transformer.wait()

## 任务 5.3：在 Amazon S3 中查看预测数据

批量转换任务将输出存储在您设置转换程序时指定的存储桶和文件夹中。您可以通过 AWS 管理控制台或笔记本在 Amazon S3 中查看预测结果。

如果您要从控制台下载和查看输出，请导航到 Amazon S3，打开以 **sagemaker-** 开头的存储桶，然后导航到位于 **/sagemaker/mlasms/batch-transform/test** 中的对象。下载 **batch_data.csv.out** 对象并使用记事本编辑器将其打开。该文件包含通过批量转换任务运行的客户记录的数百个预测值。

您还可以在笔记本中查看输出示例。


In [ ]:
!aws s3 cp --recursive $transformer.output_path ./
!head batch_data.csv.out

### 总结

恭喜！ 您已使用 Amazon SageMaker 成功运行一项批量转换任务。

### 清理

您已完成此笔记本。要进入本实验的下一部分，请执行以下操作：

- 关闭此笔记本文件。
- 返回至实验会话并继续**总结**部分。